In [9]:
!pip install pycapnp
!pip install typer
!pip install ipywidgets
import asyncio
import os

import logging
from pathlib import Path

import capnp
import typer
import zmq.asyncio
import time
from deepdrrzmq.utils.zmq_util import zmq_no_linger_context

from deepdrrzmq.utils.typer_util import unwrap_typer_param
from deepdrrzmq.utils.server_util import make_response, DeepDRRServerException, messages
import random
import string

import threading
from IPython.display import display
import ipywidgets as widgets
import time
from threading import Event
import asyncio

In [10]:
rep_port=40100
pub_port=40101
sub_port=40102

In [11]:
context = zmq.Context()
context.__enter__()

sub_socket = context.socket(zmq.SUB)
sub_socket.hwm = 10000

pub_socket = context.socket(zmq.PUB)
pub_socket.hwm = 10000

pub_socket.connect(f"tcp://localhost:{pub_port}")
sub_socket.connect(f"tcp://localhost:{sub_port}")

sub_socket.subscribe(b"/loggerd/")

In [12]:
record_button = widgets.Button(description='New Session', disabled=True, button_style='', icon='circle')
stop_button = widgets.Button(description='Stop', disabled=True, button_style='', icon='square')

session_id = widgets.Text(value='None', placeholder='', description='Session ID:', disabled=True)
record_status = widgets.Text(value='None', placeholder='', description='Recording:', disabled=True)

async def f():
    last_heard_time = 0

    while True:
        latest_msgs = {}

        try:
            for i in range(1000):
                topic, data = sub_socket.recv_multipart(flags=zmq.NOBLOCK)
                latest_msgs[topic] = data
        except zmq.ZMQError:
            pass

        for topic, data in latest_msgs.items():
            if topic == b"/loggerd/status/":
                last_heard_time = time.time()
                record_button.disabled = False
                stop_button.disabled = False
                with messages.LoggerStatus.from_bytes(data) as msg:
                    session_id.value = msg.sessionId if msg.recording else "no session"
                    record_status.value = str(msg.recording)
                    record_button.button_style = 'danger' if msg.recording else ''

        if time.time() - last_heard_time > 3:
            session_id.value = "Disconnected"
            record_status.value = "Disconnected"
            record_button.disabled = True
            stop_button.disabled = True

        await asyncio.sleep(0.1)

try:
    ui_update_task.cancel()
except:
    pass
ui_update_task = asyncio.create_task(f())

def on_record_button_clicked(b):
    pub_socket.send_multipart([b"/loggerd/start/", b""])

def on_stop_button_clicked(b):
    pub_socket.send_multipart([b"/loggerd/stop/", b""])

record_button.on_click(on_record_button_clicked)
stop_button.on_click(on_stop_button_clicked)    

display(widgets.HBox([record_button, stop_button, session_id, record_status]))

In [13]:
record_button = widgets.Button(description='New Session', disabled=True, button_style='', icon='circle')
stop_button = widgets.Button(description='Stop', disabled=True, button_style='', icon='square')

session_id = widgets.Text(value='None', placeholder='', description='Session ID:', disabled=True)
record_status = widgets.Text(value='None', placeholder='', description='Recording:', disabled=True)
async def f():
    last_heard_time = 0

    while True:
        latest_msgs = {}

        try:
            for i in range(1000):
                topic, data = sub_socket.recv_multipart(flags=zmq.NOBLOCK)
                latest_msgs[topic] = data
        except zmq.ZMQError:
            pass

        for topic, data in latest_msgs.items():
            if topic == b"/loggerd/status/":
                last_heard_time = time.time()
                record_button.disabled = False
                stop_button.disabled = False
                with messages.LoggerStatus.from_bytes(data) as msg:
                    session_id.value = msg.sessionId if msg.recording else "no session"
                    record_status.value = str(msg.recording)
                    record_button.button_style = 'danger' if msg.recording else ''

        if time.time() - last_heard_time > 3:
            session_id.value = "Disconnected"
            record_status.value = "Disconnected"
            record_button.disabled = True
            stop_button.disabled = True

        await asyncio.sleep(0.1)

try:
    ui_update_task.cancel()
except:
    pass
ui_update_task = asyncio.create_task(f())

def on_record_button_clicked(b):
    pub_socket.send_multipart([b"/loggerd/start/", b""])

def on_stop_button_clicked(b):
    pub_socket.send_multipart([b"/loggerd/stop/", b""])

record_button.on_click(on_record_button_clicked)
stop_button.on_click(on_stop_button_clicked)    

display(widgets.HBox([record_button, stop_button, session_id, record_status]))